In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv(r"C:\Users\luyil\ihealth\datasets\preproc\final_dataset.csv")

In [4]:
# Preprocessing
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['sex', 'service', 'payer']:
    data[col + '_encoded'] = le.fit_transform(data[col])

data.dtypes

Unnamed: 0                   int64
payer                       object
insurer_paid_amt_mean      float64
insurer_paid_amt_sum       float64
member_paid_amt_mean       float64
member_paid_amt_sum        float64
total_paid_amt_mean        float64
total_paid_amt_sum         float64
service_year               float64
county_code                float64
county_name                 object
age_group_code             float64
age_group_name              object
sex                         object
unique_member_cnt          float64
svc_code_ctg_desc           object
clm_cnt                    float64
member_months              float64
total_paid_amt_median      float64
insurer_paid_amt_median    float64
member_paid_amt_median     float64
service                     object
insurer_paid_sum_prop      float64
insurer_paid_mean_prop     float64
member_paid_sum_prop       float64
member_paid_mean_prop      float64
average_ops_cnt            float64
sex_encoded                  int32
service_encoded     

In [5]:
drop_cols = ["payer","service","sex","county_name","age_group_name","svc_code_ctg_desc"]
new_data = data.drop(drop_cols,axis=1)
new_data.dtypes

Unnamed: 0                   int64
insurer_paid_amt_mean      float64
insurer_paid_amt_sum       float64
member_paid_amt_mean       float64
member_paid_amt_sum        float64
total_paid_amt_mean        float64
total_paid_amt_sum         float64
service_year               float64
county_code                float64
age_group_code             float64
unique_member_cnt          float64
clm_cnt                    float64
member_months              float64
total_paid_amt_median      float64
insurer_paid_amt_median    float64
member_paid_amt_median     float64
insurer_paid_sum_prop      float64
insurer_paid_mean_prop     float64
member_paid_sum_prop       float64
member_paid_mean_prop      float64
average_ops_cnt            float64
sex_encoded                  int32
service_encoded              int32
payer_encoded                int32
dtype: object

In [6]:
# new_data.to_csv(r"C:\Users\luyil\ihealth\datasets\preproc\data_for_ml.csv")

In [7]:
import xgboost as xgb

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [9]:
X = new_data.drop(["age_group_code"],axis=1)
y = new_data["age_group_code"]
y = y - 1
#ss = StandardScaler()
#X = ss.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5360548, 23), (1340137, 23), (5360548,), (1340137,))

In [11]:
y_train.unique()

array([2., 1., 4., 0., 3.])

In [12]:
from sklearn.metrics import accuracy_score

In [16]:
X_train.head()

,Unnamed: 0,insurer_paid_amt_mean,insurer_paid_amt_sum,member_paid_amt_mean,member_paid_amt_sum,total_paid_amt_mean,total_paid_amt_sum,service_year,county_code,unique_member_cnt,...,insurer_paid_amt_median,member_paid_amt_median,insurer_paid_sum_prop,insurer_paid_mean_prop,member_paid_sum_prop,member_paid_mean_prop,average_ops_cnt,sex_encoded,service_encoded,payer_encoded
704084,716763,33.54,2247.75,4.38,293.66,37.93,2541.41,2011.0,53.0,67.0,...,19.50,0.00,0.884450,0.884260,0.115550,0.115476,1.134328,1,20,0
5690584,5823884,30.86,2129.42,46.36,3199.19,77.22,5328.61,2020.0,171.0,69.0,...,1.47,36.34,0.399620,0.399637,0.600380,0.600363,1.231884,0,24,0
2920967,2976427,178.82,5185.94,29.50,855.66,208.33,6041.60,2015.0,109.0,29.0,...,159.80,0.00,0.858372,0.858350,0.141628,0.141602,1.034483,0,15,0
5276757,5394474,33.69,673.86,14.43,288.64,48.12,962.50,2019.0,153.0,20.0,...,7.33,1.83,0.700114,0.700125,0.299886,0.299875,1.850000,0,24,1
6081660,6229777,1925.82,44294.00,0.00,0.00,1925.82,44294.00,2020.0,29.0,23.0,...,1949.00,0.00,1.000000,1.000000,0.000000,0.000000,5.521739,1,17,2


In [19]:
X_train = X_train.drop(["Unnamed: 0"],axis=1)
X_test = X_test.drop(["Unnamed: 0"],axis=1)

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 创建 DMatrix 数据

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

num_train_samples = dtrain.get_label().shape[0]
num_test_samples = dtest.get_label().shape[0]

print("训练集样本数量:", num_train_samples)
print("测试集样本数量:", num_test_samples)

# 设置参数，包括使用 GPU
params = {
    'tree_method': 'hist',  # 更新为 hist
    'device': 'cuda',       # 指定使用 cuda
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'logloss',
}

# 训练模型c
n_acc = []
bst = xgb.train(params, dtrain, num_boost_round=1000)

    # 预测
preds = bst.predict(dtest)
pred_labels = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, pred_labels)
n_acc.append(accuracy)
if accuracy>max(n_acc):
    # save model
    bst.save_model('best_model.json')
    print("Saved!")
print("Accuracy Rate:", accuracy)
print("-----------------------------------")
importance = bst.get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# 打印特征重要性
for feature, score in sorted_importance:
    print(f'Feature: {feature}, Score: {score}')

(5360548, 22) (1340137, 22) (5360548,) (1340137,)
训练集样本数量: 5360548
测试集样本数量: 1340137
Accuracy Rate: 0.5818420057053868
-----------------------------------
Feature: average_ops_cnt, Score: 31749.0
Feature: service_encoded, Score: 31601.0
Feature: county_code, Score: 29184.0
Feature: member_months, Score: 21125.0
Feature: payer_encoded, Score: 20395.0
Feature: unique_member_cnt, Score: 18996.0
Feature: total_paid_amt_median, Score: 16019.0
Feature: insurer_paid_amt_median, Score: 15861.0
Feature: service_year, Score: 14740.0
Feature: clm_cnt, Score: 12678.0
Feature: insurer_paid_amt_mean, Score: 11717.0
Feature: total_paid_amt_mean, Score: 10994.0
Feature: member_paid_amt_mean, Score: 10859.0
Feature: member_paid_amt_median, Score: 9118.0
Feature: member_paid_amt_sum, Score: 8681.0
Feature: insurer_paid_sum_prop, Score: 7798.0
Feature: insurer_paid_amt_sum, Score: 7649.0
Feature: total_paid_amt_sum, Score: 6950.0
Feature: insurer_paid_mean_prop, Score: 5587.0
Feature: sex_encoded, Score: 

In [ ]:
n_acc

[0.5483118516987442,
 0.5641826171503361,
 0.5763127202666593,
 0.5839246286014041,
 0.5923431708847677,
 0.5977530655447914,
 0.602797325945034,
 0.6070237595111545,
 0.6114083858590577]

In [ ]:
table = [
    [200,300,400,500,600,700,800,900,1000],
    n_acc
]

In [ ]:
table

[[200, 300, 400, 500, 600, 700, 800, 900, 1000],
 [0.5483118516987442,
  0.5641826171503361,
  0.5763127202666593,
  0.5839246286014041,
  0.5923431708847677,
  0.5977530655447914,
  0.602797325945034,
  0.6070237595111545,
  0.6114083858590577]]

In [ ]:
preds.shape

(1340137, 5)

In [ ]:
preds

array([[2.28444159e-01, 4.72002894e-01, 1.96731508e-01, 2.15145461e-02,
        8.13069269e-02],
       [2.80243158e-03, 5.84610701e-01, 4.02723461e-01, 7.70639582e-03,
        2.15702085e-03],
       [6.56991303e-02, 4.62868482e-01, 3.95990074e-01, 5.90264201e-02,
        1.64158866e-02],
       ...,
       [1.40571073e-01, 3.73052537e-01, 4.46786791e-01, 2.75948476e-02,
        1.19947260e-02],
       [1.07650485e-05, 2.04537809e-01, 1.67398542e-01, 2.13001236e-01,
        4.15051609e-01],
       [8.75645957e-04, 4.29841518e-01, 5.21747887e-01, 2.46262457e-02,
        2.29087248e-02]], dtype=float32)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

pred_labels = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, pred_labels)
print("Accuracy Rate:", accuracy)

Accuracy Rate: 0.5977530655447914
